## Bayesian Optimization of the lgbm model using optuna

In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import lightgbm as lgb
import optuna
from optuna.visualization import plot_parallel_coordinate, plot_slice, plot_optimization_history, \
        plot_contour, plot_param_importances

In [2]:
data = pd.read_csv('regression/final_elem_SP.csv')
data.head()

,formula,S_p,Bandgap,mpe,PF,X_b,std_BL,mean_BL,vdw_r_b,Mean_NpValence,...,Mean_NfUnfilled,AM_b,MP_b,Mean_NsUnfilled,Mean_NsValence,MeanColumn,MEN_V_b,Mean_NdUnfilled,TC_b,D_b
0,ReO3,0.336845,0.000000,0.311128,0.276212,3.44,0.000001,1.913169,1.52,3.00,...,0.0,15.99940,54.80,0.00,2.00,13.75,14.0,1.25,0.02658,0.001308
1,Yb3Mg,14.406792,0.000000,0.740480,0.573238,1.31,0.000000,3.690337,1.73,0.00,...,0.0,24.30500,923.00,0.00,2.00,2.75,14.0,0.00,160.00000,1.740000
2,B9H11,638.974583,2.911795,0.101052,0.125865,2.20,0.059786,1.236931,1.10,0.45,...,0.0,1.00794,14.01,0.55,1.45,6.40,14.1,0.00,0.18050,0.000082
3,Li3Ga7,9.061345,0.000000,0.542895,0.555130,1.81,0.073217,2.838474,1.87,0.70,...,0.0,69.72300,302.91,0.30,1.70,9.40,11.8,0.00,29.00000,5.910000
4,Sc3Cd,22.887388,0.000000,0.713049,0.719525,1.69,0.004315,3.206197,2.18,0.00,...,0.0,112.41100,594.22,0.00,2.00,5.25,13.1,6.75,97.00000,8.690000


In [3]:
data = data.drop(['TC_a','D_a','MEN_V_a','Mean_NfValence','EA_b','r_a',
'Mean_NdValence','r_b','Mean_GSbandgap','Number','Mean_GSmagmom','Mean_NfUnfilled',
'AM_b','MP_b','Mean_NsUnfilled','Mean_NsValence','MeanColumn','MEN_V_b',
'Mean_NdUnfilled','TC_b','D_b'], axis=1)

In [4]:
data.head()

,formula,S_p,Bandgap,mpe,PF,X_b,std_BL,mean_BL,vdw_r_b,Mean_NpValence,...,MeltingT,Mean_NValence,X_a,Mean_GSvolume_pa,Mean_SpaceGroupNumber,MeanCovalentRadius,Dp_a,Mean_NpUnfilled,AM_a,MP_a
0,ReO3,0.336845,0.000000,0.311128,0.276212,3.44,0.000001,1.913169,1.52,3.00,...,905.8500,9.75,1.90,10.492500,57.50,87.25,62.0000,1.50,186.207000,3459.00
1,Yb3Mg,14.406792,0.000000,0.740480,0.573238,1.31,0.000000,3.690337,1.73,0.00,...,1049.7500,12.50,1.10,31.312500,217.25,175.50,139.0000,0.00,173.040000,1097.00
2,B9H11,638.974583,2.911795,0.101052,0.125865,2.20,0.059786,1.236931,1.10,0.45,...,1064.3055,1.90,2.04,6.865875,181.40,54.85,20.5000,2.25,10.811000,2349.00
3,Li3Ga7,9.061345,0.000000,0.542895,0.555130,1.81,0.073217,2.838474,1.87,0.70,...,348.1440,9.40,0.98,18.178250,113.50,123.80,164.1125,3.50,6.941000,453.69
4,Sc3Cd,22.887388,0.000000,0.713049,0.719525,1.69,0.004315,3.206197,2.18,0.00,...,1509.0550,5.25,1.36,21.550000,194.00,163.50,97.0000,0.00,44.955912,1814.00


In [5]:
X = data.iloc[:,2:-1]
Y = data.iloc[:,1]

In [6]:
# =============================================================================
#        Define the function 
# =============================================================================
    
def objective(trial):
    data, target = X, Y
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.3)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective": "regression",
        "metric": "root_mean_squared_error",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 50),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    # pred_labels = preds
    accuracy = np.sqrt(mean_squared_error(valid_y, preds))
    return accuracy

In [8]:
# =============================================================================
#                    Perform bayesian optimization
# =============================================================================
"""
The final hyperparameters change every time the run is code. So try running at least 3-4 and select the best one
"""

f = open('regression/best-paramters_lgb_optuna_TPE_plot1.txt', 'a')
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=500)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    f.write("  Value: {}\n".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value)) 
        f.write("    {}: {}\n".format(key, value))

f.write("\n")
f.close()

[I 2023-01-08 21:39:46,888] A new study created in memory with name: no-name-71b4633b-ec9b-45a5-9549-a046543f6732
[I 2023-01-08 21:39:47,030] Trial 0 finished with value: 48.59433143065533 and parameters: {'lambda_l1': 2.658418799713775, 'lambda_l2': 3.090436805940092e-05, 'num_leaves': 47, 'feature_fraction': 0.9807412938747926, 'bagging_fraction': 0.5845058542871782, 'bagging_freq': 7, 'min_child_samples': 58, 'learning_rate': 0.04219314335777799}. Best is trial 0 with value: 48.59433143065533.
[I 2023-01-08 21:39:47,110] Trial 1 finished with value: 46.00950056825774 and parameters: {'lambda_l1': 1.741434959352254, 'lambda_l2': 0.002857333770330985, 'num_leaves': 12, 'feature_fraction': 0.9846769237191453, 'bagging_fraction': 0.9285185872048476, 'bagging_freq': 3, 'min_child_samples': 68, 'learning_rate': 0.09903824087007704}. Best is trial 1 with value: 46.00950056825774.
[I 2023-01-08 21:39:47,255] Trial 2 finished with value: 47.44932152864001 and parameters: {'lambda_l1': 0.3832

Number of finished trials: 500
Best trial:
  Value: 40.77371195316632
  Params: 
    lambda_l1: 2.7519222236869456e-05
    lambda_l2: 4.295059234532295e-08
    num_leaves: 41
    feature_fraction: 0.9699317353516802
    bagging_fraction: 0.9528809862071511
    bagging_freq: 5
    min_child_samples: 49
    learning_rate: 0.07445682930952059


In [6]:
## Parallel coordinate plot
p1 = plot_parallel_coordinate(study)
p1.write_image('images/parallel_coordinate_plot_lgb_reg.png')
p1.show()

In [7]:
p2 = optuna.visualization.plot_contour(study, params=["min_child_samples","learning_rate", "feature_fraction"])
p2.write_image('images/contour_plot_imp_lgb_reg.png')
p2.show()

In [8]:
p3 = plot_slice(study)
p3.write_image('images/slice_plot_lgb_reg.png')
p3.show()

In [9]:
p4 = plot_param_importances(study)
p4.write_image('images/param_imp_plot_lgb_reg.png')
p4.show()